# 🧠 Análisis de Nota Económica – NLP en Python

## 📌 Introducción

Este notebook presenta un análisis de lenguaje natural (NLP) aplicado a una nota publicada en el diario *La Nación* el **29 de junio de 2025**, escrita por **José Del Río**. El artículo se enfoca en el estado de ánimo y las preocupaciones de distintos referentes empresariales frente a la coyuntura económica argentina.

A través del uso de herramientas de NLP en Python, exploraremos el contenido de la nota para extraer patrones, reducir el texto a sus componentes esenciales y medir su carga emocional o polaridad.

## 🎯 Objetivos

- Preprocesar el texto mediante tokenización, lematización y remoción de stopwords.
- Extraer tokens relevantes del artículo y visualizar palabras frecuentes.
- Realizar un análisis de sentimientos o polaridad del texto completo.
- Observar las distintas opiniones expresadas por 5 referentes clave:
  - **Javier Bolzico**, presidente de Adeba (sector bancario).
  - **Gustavo Weiss**, titular de Camarco (infraestructura).
  - **Issel Kiperszmid**, vicepresidente de la CEDU (desarrollos inmobiliarios).
  - **Martín Rapallini**, presidente de la Unión Industrial Argentina (industria nacional).
  - **Mario Grinman**, titular de la Cámara Argentina de Comercio (comercio y servicios).

Estos representantes, junto con la Bolsa y la Sociedad Rural Argentina, forman un termómetro de los sectores más relevantes del país.

## 🔗 Fuente del artículo

[Plan económico: ¿cuál es el verdadero riesgo que preocupa a los empresarios? (La Nación)](https://www.lanacion.com.ar/economia/plan-economico-cual-es-el-verdadero-riesgo-que-preocupa-a-los-empresarios-nid29062025/)

## 🧰 Librerías necesarias

Este proyecto utilizará las siguientes bibliotecas de Python:

- **spaCy**: Para tokenización y lematización en idioma español.
- **nltk**: Para eliminación de *stopwords*.
- **textblob** y **textblob-es**: Para análisis de sentimientos.
- **matplotlib**: Para visualización gráfica de los resultados.
- **seaborn**: Para visualizaciones estadísticas mejoradas.
- **pandas**: Para manipulación y análisis de datos tabulares.
- **numpy**: Para operaciones numéricas y estructuras de datos eficientes.
- **vaderSentiment** *(opcional)*: Análisis de polaridad si se decide traducir el texto a inglés.

In [ ]:
# Procesamiento de lenguaje natural
import spacy
import nltk
from nltk.corpus import stopwords

# Descargar stopwords de NLTK
nltk.download('stopwords')

# Análisis de sentimientos
from textblob import TextBlob

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Análisis y manipulación de datos
import pandas as pd
import numpy as np

# VADER (opcional)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Descargar modelo de spaCy en español si aún no está cargado
nlp = spacy.load("es_core_news_sm")

# Descargar los corpora necesarios para textblob desde consola
!python -m textblob.download_corpora

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews 

## 📥 Carga del texto

Para trabajar con el contenido del artículo periodístico, primero cargamos el texto desde un archivo local llamado `economic_article-2025-06-29.txt`. Este archivo contiene la nota completa publicada en el diario *La Nación* el 29 de junio de 2025, escrita por José Del Río.

El archivo debe estar ubicado en el mismo directorio que este notebook o se debe indicar la ruta relativa correspondiente.

Una vez cargado, el contenido se guarda en la variable `texto` para su posterior análisis.

In [2]:
# Leer el texto desde el archivo .txt
with open("economic_article-2025-06-29.txt", "r", encoding="utf-8") as file:
    texto = file.read()

# Ver los primeros 500 caracteres del texto para verificar
print(texto[:500])

Plan económico. Cuál es el verdadero riesgo que preocupa a los empresarios?
El debate es cada vez más explícito; la brecha entre la macro y la micro está a flor de piel; cuáles son las reformas que esperan y porqué tienen la mira en el empleo y la actividad; cuál es el país que corre por las venas de quienes hacen negocios en la Argentina desde hace muchos años; las expectativas sobre el dólar y el dilema de la competitividad.
Fueron tres cafés negros y un cortado. La charla se extendió por 60 m


## 🧹 Preprocesamiento del texto

Antes de realizar cualquier análisis, es necesario limpiar y estructurar el texto para facilitar su interpretación computacional. En este paso aplicamos varias técnicas de procesamiento de lenguaje natural (NLP):

- **Conversión a minúsculas**: Unificamos todo el texto para evitar diferencias entre palabras idénticas por su uso de mayúsculas.

- **Tokenización**: Dividimos el texto en unidades lingüísticas mínimas llamadas *tokens* (generalmente palabras).

- **Lematización**: Reducimos cada palabra a su forma base o *lema* (por ejemplo, "tenían" → "tener").

- **Eliminación de stopwords**: Quitamos palabras vacías o muy frecuentes que no aportan valor semántico al análisis (como “el”, “y”, “de”).

- **Filtro de puntuación y tokens no alfabéticos**: Se descartan signos de puntuación, números y palabras no alfabéticas para centrarnos en contenido relevante.

El resultado de este proceso es una lista de palabras limpias y lematizadas que nos servirá como base para el análisis de frecuencia y sentimiento.

### Tokenización

In [ ]:
# Procesar el texto
doc = nlp(texto)

print("✅ Texto procesado con spaCy")
print("Ejemplo de tokens:")
print([token.text for token in doc[:20]])

✅ Texto procesado con spaCy
Ejemplo de tokens:
['Plan', 'económico', '.', 'Cuál', 'es', 'el', 'verdadero', 'riesgo', 'que', 'preocupa', 'a', 'los', 'empresarios', '?', '\n', 'El', 'debate', 'es', 'cada', 'vez']


### Eliminar Signos de Puntuación

In [12]:
# Extraer solo los tokens que son palabras (sin puntuación, números, etc.)
tokens = [token for token in doc if token.is_alpha]

print("✅ Tokens alfabéticos extraídos:")
print([t.text for t in tokens[:20]])

✅ Tokens alfabéticos extraídos:
['Plan', 'económico', 'Cuál', 'es', 'el', 'verdadero', 'riesgo', 'que', 'preocupa', 'a', 'los', 'empresarios', 'El', 'debate', 'es', 'cada', 'vez', 'más', 'explícito', 'la']


### Convertir a Minúsculas

In [13]:
tokens_minuscula = [token.text.lower() for token in tokens]

print("✅ Tokens en minúscula:")
print(tokens_minuscula[:20])

✅ Tokens en minúscula:
['plan', 'económico', 'cuál', 'es', 'el', 'verdadero', 'riesgo', 'que', 'preocupa', 'a', 'los', 'empresarios', 'el', 'debate', 'es', 'cada', 'vez', 'más', 'explícito', 'la']


### Lematización

In [14]:
tokens_lematizados = [token.lemma_ for token in tokens]

print("✅ Tokens lematizados:")
print(tokens_lematizados[:20])

✅ Tokens lematizados:
['Plan', 'económico', 'cuál', 'ser', 'el', 'verdadero', 'riesgo', 'que', 'preocupar', 'a', 'el', 'empresario', 'el', 'debate', 'ser', 'cada', 'vez', 'más', 'explícito', 'el']


### Eliminado de Stop Words

In [15]:
stop_words = set(stopwords.words('spanish'))

# Usamos los lemas en minúscula
tokens_finales = [lemma.lower() for lemma in tokens_lematizados if lemma.lower() not in stop_words]

print("✅ Tokens limpios (lematizados, minúscula, sin stopwords):")
print(tokens_finales[:50])

✅ Tokens limpios (lematizados, minúscula, sin stopwords):
['plan', 'económico', 'cuál', 'ser', 'verdadero', 'riesgo', 'preocupar', 'empresario', 'debate', 'ser', 'cada', 'vez', 'explícito', 'brecha', 'macro', 'micro', 'flor', 'piel', 'cuál', 'ser', 'reforma', 'esperar', 'porqué', 'tener', 'mira', 'empleo', 'actividad', 'cuál', 'ser', 'país', 'correr', 'vena', 'hacer', 'negocio', 'argentina', 'hacer', 'año', 'expectativa', 'dólar', 'dilema', 'competitividad', 'ser', 'tres', 'café', 'negro', 'cortado', 'charla', 'extender', 'minuto', 'eufemismo']


## ❤️ Análisis de Sentimientos en Español con PySentimiento

El análisis de sentimientos permite identificar si un texto expresa emociones **positivas**, **negativas** o **neutras**.

En este proyecto utilizamos:

- **PySentimiento**: una librería basada en modelos BERT entrenados en español. Permite realizar análisis de sentimientos sin necesidad de traducir el texto. Ideal para evaluar discursos, noticias o declaraciones en su idioma original.

Podemos aplicar este análisis a:
- El artículo completo
- Fragmentos por párrafo o por referente (ej: opiniones de Rapallini, Grinman, Bolzico, etc.)

Esto nos ayuda a detectar diferencias en el tono emocional según el sector económico y la perspectiva del actor entrevistado.

In [1]:
from pysentimiento import create_analyzer

# Crear el analizador para español
analyzer = create_analyzer(task="sentiment", lang="es")

# Ejemplo simple
resultado = analyzer.predict(tokens_finales)

print("🔍 Sentimiento detectado:", resultado.output)
print("📊 Probabilidades:", resultado.probas)


ModuleNotFoundError: No module named 'pysentimiento'

In [ ]:
# Suponiendo que ya cargaste el artículo completo en la variable 'texto'
parrafos = texto.split('\n')

for i, parrafo in enumerate(parrafos):
    if parrafo.strip():
        resultado = analyzer.predict(parrafo.strip())
        print(f"🧾 Párrafo {i+1}:")
        print(f"   📌 Texto: {parrafo[:100]}...")
        print(f"   🔍 Sentimiento: {resultado.output}")
        print(f"   📊 Probas: {resultado.probas}")
        print()

